In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0

In [3]:
N_centr = 10
N_real_centr = 5
eIDbins = [1,2,3,4,5]
primaryID = [[18,19,20,23,24,25],[19,20,24,25],[20,25]]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,20,40,40,60,60,80,80,93]
real_file_path = 'input/'
real_file_names=['my-m_ee_Run14AuAu_45th_new_19617_725runs.root']#m_ee_Run14AuAu_45th_new_19617_240runs , m_ee_Run14AuAu_44th_new_19614_975runs
sim_file_path="../sim/output/Newembed/sngl/"
primary_file_names=['phi_50M_v6.root']
hadron_file_names=['charged_pi_100M_v1.root']
dalitz_file_names=['dalitz_5M_v3.root']
conv_file_names=['conv_100M.root']
sim_file_names = [primary_file_names,hadron_file_names,dalitz_file_names,conv_file_names ]
sim_file_names = [dalitz_file_names, hadron_file_names, primary_file_names ]
part_names = ["c#bar{c}","dalitz","#gamma #rightarrow ee","#pi^{-}"]
eID_names = ["BDT loose","tight+BDT","BDT loose","BDT basic","BDT tight"]
hist_names = ["el_pt_hist_"]

In [4]:
hists_read_real, hists_read_sim = [], []

for iFilereal in range(len(real_file_names)):
    infile = root.TFile.Open(real_file_path+real_file_names[iFilereal], "read")
    hists_read_real_ifile = []
    for icentr in range(N_centr):
        hists_read_real_ifile.append(infile.Get(hist_names[0]+f'{icentr}'))
        hists_read_real_ifile[-1].SetDirectory(root.nullptr)
        hists_read_real_ifile[-1].SetName(hist_names[0]+f'{icentr}'+'_real')
    hists_read_real.append(hists_read_real_ifile)
    infile.Close()

for iFilesim in range(len(sim_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFilesim][0], "read")
    hists_read_sim_ifile = []
    for icentr in range(N_centr):
        hists_read_sim_ifile.append(infile.Get(hist_names[0]+f'{icentr}'))
        hists_read_sim_ifile[-1].SetDirectory(root.nullptr)
        hists_read_sim_ifile[-1].SetName(hist_names[0]+f'{icentr}'+f'_sim{iFilesim}_')
    hists_read_sim.append(hists_read_sim_ifile)
    infile.Close()

In [5]:
hists_pt_real, hists_pt_sim = [], []

for iFilereal in range(len(real_file_names)):
    hists_pt_real_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_real_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_real_centr = []
            for icentr in range(N_centr):
                hists_pt_real_centr.append(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_pr{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_real_centr[-1].Add(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_pr{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_real_conv.append(hists_pt_real_centr)
        hists_pt_real_eid.append(hists_pt_real_conv)
    hists_pt_real.append(hists_pt_real_eid)
    

for iFilesim in range(len(sim_file_names)):
    hists_pt_sim_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_sim_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_sim_centr = []
            for icentr in range(N_centr):
                hists_pt_sim_centr.append(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_pr{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_sim_centr[-1].Add(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_pr{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_sim_conv.append(hists_pt_sim_centr)
        hists_pt_sim_eid.append(hists_pt_sim_conv)
    hists_pt_sim.append(hists_pt_sim_eid)

In [6]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
h = []
legends = []
iED = 4
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.1, 4.9))
    h[-1].SetMaximum(1.2* max(hists_pt_real[0][iED][0][icentr].GetMaximum(),hists_pt_real[0][iED][0][icentr+5].GetMaximum()))
    h[-1].SetMinimum(10)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    
    Format_Hist_total(hists_pt_real[0][iED][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    Format_Hist_total(hists_pt_real[0][iED][0][icentr+5], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=4,  Lwidth=4,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    root.gPad.SetLogy()
    h[-1].Draw()
    hists_pt_real[0][iED][0][icentr].Draw("same")
    hists_pt_real[0][iED][0][icentr+5].Draw("same")
    legends.append(root.TLegend(0.5,0.7,0.9,0.9))
    legends[-1].AddEntry(hists_pt_real[0][iED][0][icentr],"pos","l")
    legends[-1].AddEntry(hists_pt_real[0][iED][0][icentr+5],"neg","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
c0.Draw()

In [7]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
ipart = 0
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.1, 4.9))
    h[-1].SetMaximum(1.2* max(hists_pt_sim[ipart][0][0][icentr].GetMaximum(),hists_pt_sim[ipart][0][0][icentr+5].GetMaximum()))
    h[-1].SetMinimum(100)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr+5], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=4,  Lwidth=4,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    root.gPad.SetLogy()
    h[-1].Draw()
    hists_pt_sim[ipart][0][0][icentr].Draw("same")
    hists_pt_sim[ipart][0][0][icentr+5].Draw("same")
    legends.append(root.TLegend(0.5,0.7,0.9,0.9))
    legends[-1].AddEntry(hists_pt_sim[ipart][0][0][icentr],"pos","l")
    legends[-1].AddEntry(hists_pt_sim[ipart][0][0][icentr+5],"neg","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
c0.Draw()

In [8]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
ipart = 1
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.1, 4.9))
    h[-1].SetMaximum(1.2* max(hists_pt_sim[ipart][0][0][icentr].GetMaximum(),hists_pt_sim[ipart][0][0][icentr+5].GetMaximum()))
    h[-1].SetMinimum(100)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr+5], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=4,  Lwidth=4,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    root.gPad.SetLogy()
    h[-1].Draw()
    hists_pt_sim[ipart][0][0][icentr].Draw("same")
    hists_pt_sim[ipart][0][0][icentr+5].Draw("same")
    legends.append(root.TLegend(0.5,0.7,0.9,0.9))
    legends[-1].AddEntry(hists_pt_sim[ipart][0][0][icentr],"pos","l")
    legends[-1].AddEntry(hists_pt_sim[ipart][0][0][icentr+5],"neg","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
c0.Draw()

In [9]:
idivide = 1

In [10]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
    h[-1].SetMaximum(2)
    h[-1].SetMinimum(0.5)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    h[-1].Draw()
    legends.append(root.TLegend(0.5,0.7,0.9,0.9)) 
    Format_Legend(legends[-1])
    for eID in range(1,len(eIDbins)):
        Format_Hist_total(hists_pt_real[0][eID][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=colors[eID],  Lwidth=6,  Lcolor=colors[eID],  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        if idivide : hists_pt_real[0][eID][0][icentr].Divide(hists_pt_real[0][eID][0][icentr+5])
        hists_pt_real[0][eID][0][icentr].Draw("same")
        legends[-1].AddEntry(hists_pt_real[0][eID][0][icentr],f"{eID_names[eID]}","l")
    legends[-1].Draw()

c0.Draw()

In [11]:
iFile = 0
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
    h[-1].SetMaximum(2)
    h[-1].SetMinimum(0.5)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    h[-1].Draw()
    legends.append(root.TLegend(0.5,0.7,0.9,0.9)) 
    Format_Legend(legends[-1])
    for eID in range(1,len(eIDbins)):
        Format_Hist_total(hists_pt_sim[iFile][eID][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=colors[eID],  Lwidth=6,  Lcolor=colors[eID],  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        if idivide : hists_pt_sim[iFile][eID][0][icentr].Divide(hists_pt_sim[iFile][eID][0][icentr+5])
        hists_pt_sim[iFile][eID][0][icentr].Draw("same")
        legends[-1].AddEntry(hists_pt_sim[iFile][eID][0][icentr],f"{eID_names[eID]}","l")
    legends[-1].Draw()

c0.Draw()

In [12]:
iFile = 1
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
    h[-1].SetMaximum(2)
    h[-1].SetMinimum(0.5)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    h[-1].Draw()
    legends.append(root.TLegend(0.5,0.7,0.9,0.9)) 
    Format_Legend(legends[-1])
    for eID in range(1,len(eIDbins)):
        Format_Hist_total(hists_pt_sim[iFile][eID][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=colors[eID],  Lwidth=6,  Lcolor=colors[eID],  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        if idivide : hists_pt_sim[iFile][eID][0][icentr].Divide(hists_pt_sim[iFile][eID][0][icentr+5])
        hists_pt_sim[iFile][eID][0][icentr].Draw("same")
        legends[-1].AddEntry(hists_pt_sim[iFile][eID][0][icentr],f"{eID_names[eID]}","l")
    legends[-1].Draw()

c0.Draw()

In [13]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
    h[-1].SetMaximum(2)
    h[-1].SetMinimum(0.5)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    h[-1].Draw()
    legends.append(root.TLegend(0.5,0.7,0.9,0.9)) 
    Format_Legend(legends[-1])
    for eID in range(0,len(eIDbins)):
        Format_Hist_total(hists_pt_real[0][eID][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=colors[eID],  Lwidth=6,  Lcolor=colors[eID],  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        if idivide : hists_pt_real[0][eID][0][icentr].Divide(hists_pt_sim[0][eID][0][icentr])
        hists_pt_real[0][eID][0][icentr].Draw("same")
        legends[-1].AddEntry(hists_pt_real[0][eID][0][icentr],f"{eID_names[eID]}","l")
    legends[-1].Draw()

c0.Draw()
idivide = 0

In [14]:
if False:
    c0 = root.TCanvas(f"c0",f"c0",1400,1400)
    c0.Divide(2,3)
    for icentr in range(N_centr):
        c0.cd(icentr+1)
        root.gPad.SetLogy()
        hists_pt_sim[1][0][0][icentr].Draw()
    c0.Draw()

signal = electrons + hadron => 1 = (1-x) + x

applying cut: signal, electrons and hadraons are reduced diffrently

signal_fraction = electron_fraction * (1-x) + hadron_fraction * x

x = ( electron_fraction - signal_fraction ) / ( electron_fraction - hadron_fraction )

In [15]:
rebin = 1
for icentr in range(N_centr):
    for ieID in range(0,len(eIDbins)):
        for iconv in range(len(primaryID)):
            hists_pt_real[0][ieID][iconv][icentr].Rebin(rebin)
            hists_pt_sim[0][ieID][iconv][icentr].Rebin(rebin)
            hists_pt_sim[1][ieID][iconv][icentr].Rebin(rebin)

In [16]:
graphs= [] 
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(1,len(eIDbins)):
        graphs_eid = []
        for iconv in range(len(primaryID)):
            graphs_conv = []
            for iptbin in range(1,1+hists_pt_real[0][ieID][iconv][icentr].GetNbinsX()):
                raw_signal_frac =  hists_pt_real[0][ieID][iconv][icentr].GetBinContent(iptbin)/(hists_pt_real[0][0][iconv][icentr].GetBinContent(iptbin)+1e-6)
                raw_primary_frac = hists_pt_sim[0][ieID][iconv][icentr].GetBinContent(iptbin)/(hists_pt_sim[0][0][iconv][icentr].GetBinContent(iptbin)+1e-6)
                raw_hadron_frac =  hists_pt_sim[1][ieID][iconv][icentr].GetBinContent(iptbin)/(hists_pt_sim[1][0][iconv][icentr].GetBinContent(iptbin)+1e-6)
                #print(raw_signal_frac, raw_primary_frac, raw_hadron_frac)
                if (raw_primary_frac - raw_signal_frac) == 0: hadron_frac = 0
                else: 
                    if raw_primary_frac - raw_hadron_frac > 0:
                        hadron_frac = ( raw_primary_frac - raw_signal_frac ) / ( raw_primary_frac - raw_hadron_frac )
                    else: hadron_frac = -1
                    if hadron_frac<0: hadron_frac = 0
                    if hadron_frac>1: hadron_frac = 1
                hadron_frac_err = hadron_frac * (hists_pt_sim[1][ieID][iconv][icentr].GetBinError(iptbin)/(hists_pt_sim[1][ieID][iconv][icentr].GetBinContent(iptbin)+1e-6))*2**0.5
                    #print(hadron_frac)
                graphs_conv.append([hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin),hadron_frac,hadron_frac_err])
            graphs_eid.append(graphs_conv)
        graphs_centr.append(graphs_eid)
    graphs.append(graphs_centr)
print(graphs)

[[[[[0.05, 0, 0.0], [0.15000000000000002, 1.0, 0.0], [0.25, 1.0, 0.0], [0.35000000000000003, 1, 2.449482394335995], [0.45, 0.8772773819559366, 1.4857911263850425], [0.55, 0.6110433558852842, 1.211201776243237], [0.6500000000000001, 0.06245035570653563, 0.1149911797421214], [0.7500000000000001, 0, 0.0], [0.8500000000000001, 0, 0.0], [0.9500000000000001, 0, 0.0], [1.05, 0, 0.0], [1.1500000000000001, 0, 0.0], [1.2500000000000002, 0, 0.0], [1.35, 0, 0.0], [1.4500000000000002, 0, 0.0], [1.55, 0, 0.0], [1.6500000000000001, 0, 0.0], [1.7500000000000002, 0, 0.0], [1.85, 0, 0.0], [1.9500000000000002, 0, 0.0], [2.05, 0, 0.0], [2.15, 0, 0.0], [2.25, 0, 0.0], [2.35, 0, 0.0], [2.45, 0, 0.0], [2.55, 0, 0.0], [2.65, 0, 0.0], [2.75, 0, 0.0], [2.85, 0, 0.0], [2.95, 0, 0.0], [3.05, 0, 0.0], [3.15, 0, 0.0], [3.25, 0, 0.0], [3.35, 0, 0.0], [3.45, 0, 0.0], [3.55, 0, 0.0], [3.65, 0, 0.0], [3.75, 0, 0.0], [3.85, 0, 0.0], [3.95, 0, 0.0], [4.05, 0, 0.0], [4.15, 0, 0.0], [4.25, 0, 0.0], [4.35, 0, 0.0], [4.45, 0

In [17]:
Tgraphs= [] 
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(0,len(eIDbins)-1):
        graphs_eid = []
        for iconv in range(len(primaryID)):
            graphs_conv = root.TGraphErrors()
            ipoint = 0
            for iptbin in range(0,0+hists_pt_real[0][ieID][iconv][icentr].GetNbinsX()):
                if hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin+1)<0.4: continue
                if hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin+1)>2.0: continue
                graphs_conv.AddPoint(hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin+1),graphs[icentr][ieID][iconv][iptbin][1])
                graphs_conv.SetPointError(ipoint,hists_pt_real[0][ieID][iconv][icentr].GetBinWidth(iptbin+1)/2,graphs[icentr][ieID][iconv][iptbin][2])
                ipoint+=1

            graphs_eid.append(graphs_conv)
        graphs_centr.append(graphs_eid)
    Tgraphs.append(graphs_centr)

In [18]:
c0 = root.TCanvas(f"c0",f"c0",1400,1400*2)
c0.Divide(3,6)
h = []
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(0,len(eIDbins)-1):
        graphs_eid = []
        for iconv in range(len(primaryID)):
            c0.cd(icentr+1+6*iconv)
            if ieID == 0 :
                h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
                h[-1].SetMaximum(0.19)
                Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                                      Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
                h[-1].Draw()
                Format_Graph( Tgraphs[icentr][ieID][iconv],  Mstyle=21,  Msize=1,  Mcolor=1,  Lwidth=1,  Lcolor=1,  Malpha=1,  Lalpha=1)
                Tgraphs[icentr][ieID][iconv].Draw("same P")
            else :
                Format_Graph( Tgraphs[icentr][ieID][iconv],  Mstyle=20,  Msize=1,  Mcolor=2,  Lwidth=1,  Lcolor=1,  Malpha=2,  Lalpha=1)
                Tgraphs[icentr][ieID][iconv].Draw("same P")
c0.Draw()

In [19]:
hists_pt_real, hists_pt_sim = [], []

for iFilereal in range(len(real_file_names)):
    hists_pt_real_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_real_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_real_centr = []
            for icentr in range(N_centr):
                hists_pt_real_centr.append(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_p1r{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_real_centr[-1].Add(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_p1r{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_real_conv.append(hists_pt_real_centr)
        hists_pt_real_eid.append(hists_pt_real_conv)
    hists_pt_real.append(hists_pt_real_eid)
    

for iFilesim in range(len(sim_file_names)):
    hists_pt_sim_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_sim_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_sim_centr = []
            for icentr in range(N_centr):
                hists_pt_sim_centr.append(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_p1r{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_sim_centr[-1].Add(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_p1r{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_sim_conv.append(hists_pt_sim_centr)
        hists_pt_sim_eid.append(hists_pt_sim_conv)
    hists_pt_sim.append(hists_pt_sim_eid)

In [20]:
graphs= [] 
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(1,len(eIDbins)):
        graphs_eid = []
        for iconv in range(len(primaryID)):
            raw_signal_frac =  hists_pt_real[0][ieID][iconv][icentr].Integral(5,10)/(hists_pt_real[0][0][iconv][icentr].Integral(5,10)+1e-6)
            raw_primary_frac = hists_pt_sim[0][ieID][iconv][icentr].Integral(5,10)/(hists_pt_sim[0][0][iconv][icentr].Integral(5,10)+1e-6)
            raw_hadron_frac =  hists_pt_sim[1][ieID][iconv][icentr].Integral(5,10)/(hists_pt_sim[1][0][iconv][icentr].Integral(5,10)+1e-6)
            #print(raw_signal_frac, raw_primary_frac, raw_hadron_frac)
            if (raw_signal_frac - raw_primary_frac) == 0: hadron_frac = 0
            else: 
                if raw_primary_frac - raw_hadron_frac > 0:
                    hadron_frac = ( raw_primary_frac - raw_signal_frac ) / ( raw_primary_frac - raw_hadron_frac )
                else: hadron_frac = -1
                if hadron_frac<0: hadron_frac = 0
                if hadron_frac>1: hadron_frac = 1
            graphs_eid.append(hadron_frac)
        graphs_centr.append(graphs_eid)
    graphs.append(graphs_centr)
print(graphs)

[[[0, 0, 0], [0, 0, 0], [0.17494924409620724, 0.17830785115867948, 0.16987196733170487], [0.7305937216915109, 0.7406511011311431, 0.7320421536045933]], [[0, 0, 0], [0, 0, 0], [0.2558627117937791, 0.27316894418145904, 0.32351741079465296], [1, 1, 1]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0.14105786788615282, 0.15109631287312392, 0.15572402941592706], [0.7454385675464309, 0.7499810617817739, 0.7580300998427545]], [[0, 0, 0], [0, 0, 0], [0.25679219766133365, 0.26014860578743454, 0.26235136007152], [1, 1, 1]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]]


In [21]:
if True :
    c0 = root.TCanvas(f"c0",f"c0",1400,1400*2)
    c0.Divide(3,6)
    for icentr in range(N_centr):
        for iconv in range(len(primaryID)):
            c0.cd(icentr+1+6*iconv)
            root.gPad.SetLogy()
                
            Format_Hist_total(hists_pt_real[0][0][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=2, Mcolor=2,  Lwidth=4,  Lcolor=2,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
            hists_pt_real[0][0][iconv][icentr].Draw()
    
            for iptbin in range(1,1+hists_pt_real[0][0][iconv][icentr].GetNbinsX()):
                hists_pt_sim[1][0][iconv][icentr].SetBinContent(iptbin,graphs[icentr][0][iconv]*hists_pt_real[0][0][iconv][icentr].GetBinContent(iptbin))
                hists_pt_sim[1][1][iconv][icentr].SetBinContent(iptbin,graphs[icentr][1][iconv]*hists_pt_real[0][0][iconv][icentr].GetBinContent(iptbin))
                
            Format_Hist_total(hists_pt_sim[1][0][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
            Format_Hist_total(hists_pt_sim[1][1][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=0, Mcolor=4,  Lwidth=3,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
            hists_pt_sim[1][0][iconv][icentr].Draw('same H')
            hists_pt_sim[1][1][iconv][icentr].Draw('same H')
    c0.Draw()            